In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pycep_correios
import geopy
import pycep_correios
from pycep_correios import get_address_from_cep, WebService
from geopy.geocoders import Nominatim

In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# conexão com o banco de dados SQL Server
engine = create_engine(os.environ.get('DATABASE_URL_SQL_SERVER'), echo=False)

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
geolocator = Nominatim(user_agent=user_agent)

In [3]:
# municípios brasileiros, tabela IBGE importada da fonte de dados
# https://www.ibge.gov.br/explica/codigos-dos-municipios.php
sql = '''
SELECT 
       *
  FROM [dbo].[ibge_municipios_brasil]
'''

df_ibge = pd.read_sql(sql, engine)

# cria um campo com o código do município sem o último dígito
df_ibge['IBGE'] = df_ibge['Código Município Completo'].astype(str).str[:-1]

df_ibge.head()

,index,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município,IBGE
0,0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,110001
1,1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,110037
2,2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,110040
3,3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,110034
4,4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,110002


In [4]:

sql = '''
SELECT [CNES]
      ,[UF]
      ,[IBGE]
      ,[NOME]
      ,[LOGRADOURO]
      ,[BAIRRO]
      ,[LATITUDE]
      ,[LONGITUDE]
  FROM [dbo].[cadastro_estabelecimentos_cnes]
'''

df_cnes = pd.read_sql(sql, engine)
df_cnes['IBGE'] = df_cnes['IBGE'].astype(str)
df_cnes.head()

,CNES,UF,IBGE,NOME,LOGRADOURO,BAIRRO,LATITUDE,LONGITUDE
0,10162,50,500270,SESAU UBS DR IVAN HIDELBRAND DA COSTA BURITI,RUA JOSE SOARES DE ARAUJO,BURITI,-20.49071,-54.67119
1,10170,50,500270,SESAU UBS DR VESPASIANO BARBOSA MARTINS VILA P...,RUA DALILA SIQUEIRA,VILA POPULAR,-20.45282,-54.70452
2,10189,50,500270,SESAU UBS DR NICOLAU FRAGELLI LAR DO TRABALHADOR,RUA DOS NARCISOS,LAR DO TRABALHADOR,-20.45192,-54.64448
3,10197,50,500270,SESAU USF DR SUMIE IKEDA RODRIGUES SERRADINHO,RUA DELMIRO GOUVEIA,VILA SERRADINHO,-20.45873,-54.68441
4,10200,50,500270,SESAU USF DR HIROSE ADANIA BONANCA,RUA JARDIM SANTO INACIO,BONANCA,-20.49342,-54.66122


In [5]:
# concatena o código IBGE com o dataframe de municípios
df_cnes = df_cnes.merge(df_ibge, how='left', on='IBGE')
df_cnes.head()

,CNES,UF_x,IBGE,NOME,LOGRADOURO,BAIRRO,LATITUDE,LONGITUDE,index,UF_y,...,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,10162,50,500270,SESAU UBS DR IVAN HIDELBRAND DA COSTA BURITI,RUA JOSE SOARES DE ARAUJO,BURITI,-20.49071,-54.67119,5122.0,50.0,...,Campo Grande,500001.0,Campo Grande,2.0,Centro Norte de Mato Grosso do Sul,4.0,Campo Grande,2704.0,5002704.0,Campo Grande
1,10170,50,500270,SESAU UBS DR VESPASIANO BARBOSA MARTINS VILA P...,RUA DALILA SIQUEIRA,VILA POPULAR,-20.45282,-54.70452,5122.0,50.0,...,Campo Grande,500001.0,Campo Grande,2.0,Centro Norte de Mato Grosso do Sul,4.0,Campo Grande,2704.0,5002704.0,Campo Grande
2,10189,50,500270,SESAU UBS DR NICOLAU FRAGELLI LAR DO TRABALHADOR,RUA DOS NARCISOS,LAR DO TRABALHADOR,-20.45192,-54.64448,5122.0,50.0,...,Campo Grande,500001.0,Campo Grande,2.0,Centro Norte de Mato Grosso do Sul,4.0,Campo Grande,2704.0,5002704.0,Campo Grande
3,10197,50,500270,SESAU USF DR SUMIE IKEDA RODRIGUES SERRADINHO,RUA DELMIRO GOUVEIA,VILA SERRADINHO,-20.45873,-54.68441,5122.0,50.0,...,Campo Grande,500001.0,Campo Grande,2.0,Centro Norte de Mato Grosso do Sul,4.0,Campo Grande,2704.0,5002704.0,Campo Grande
4,10200,50,500270,SESAU USF DR HIROSE ADANIA BONANCA,RUA JARDIM SANTO INACIO,BONANCA,-20.49342,-54.66122,5122.0,50.0,...,Campo Grande,500001.0,Campo Grande,2.0,Centro Norte de Mato Grosso do Sul,4.0,Campo Grande,2704.0,5002704.0,Campo Grande


In [6]:
itens_saida = []

for index, row in df_cnes.iterrows():
    query = f"{row['LOGRADOURO']}, {row['BAIRRO']}, {row['Nome_Município']}, {row['UF_x']}"
    location = geolocator.geocode(query)
    if location is None:
        print('Localização não encontrada', row['LOGRADOURO'], row['Nome_Município'])

    if location is not None:
        print(row)
        print('Logradouro', row['LOGRADOURO'])
        print('Munícipio', row['Nome_Município'])
        print(location)
        row['latitude_geolocator'] = location.latitude
        row['longitude_geolocator'] = location.longitude
        print(location.latitude, location.longitude)
        itens_saida.append(row)

df_saida = pd.DataFrame(itens_saida)
df_saida.head()

df_saida.to_sql('cadastro_estabelecimentos_cnes_com_geolocator', engine, if_exists='replace', chunksize=500, schema='dbo')
print(f'{len(df_saida)} registros salvos no banco de dados com sucesso.')

CNES                                                                           10162
UF_x                                                                              50
IBGE                                                                          500270
NOME                                    SESAU UBS DR IVAN HIDELBRAND DA COSTA BURITI
LOGRADOURO                                                 RUA JOSE SOARES DE ARAUJO
BAIRRO                                                                        BURITI
LATITUDE                                                                   -20.49071
LONGITUDE                                                                  -54.67119
index                                                                         5122.0
UF_y                                                                            50.0
Nome_UF                                                           Mato Grosso do Sul
Região Geográfica Intermediária                                  

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=RUA+CHICO+BUQUIRA%2C+EUGENIO+DE+MELO%2C+S%C3%A3o+Jos%C3%A9+dos+Campos%2C+35&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
sql = '''
SELECT 
       [UF]
      ,[MUNICIPIO]
      ,[ESTABELECIMENTO]
      ,[LOGRADOURO]
      ,[NUMERO]
      ,[BAIRRO]
  FROM [dbo].[unidades_vacinacao]
'''
df = pd.read_sql(sql, engine)
df.head()


,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,None,ZONA RURAL


In [ ]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
geolocator = Nominatim(user_agent=user_agent)
for index, row in df.head(5).iterrows():
    location = geolocator.geocode(row['LOGRADOURO'] + ", " + row['MUNICIPIO'])
    if location is None:
        print('Localização não encontrada', row['LOGRADOURO'], row['MUNICIPIO'])

    if location is not None:
        print('Logradouro', row['LOGRADOURO'])
        print('Munícipio', row['MUNICIPIO'])  
        print(location)
        print(location.latitude, location.longitude)        

Localização não encontrada RUA 07 QUEDAS ACRELANDIA
Logradouro BR 364 KM 114
Munícipio ACRELANDIA
Estrada do Pacífico, Colonia Oito de Maio, Acrelândia, Região Geográfica Imediata de Rio Branco, Região Geográfica Intermediária de Rio Branco, Acre, Região Norte, Brasil
-9.8106663 -66.8605533
Logradouro AVENIDA PARANA
Munícipio ACRELANDIA
Avenida Parana, Acrelândia, Região Geográfica Imediata de Rio Branco, Região Geográfica Intermediária de Rio Branco, Acre, Região Norte, Brasil
-10.0781291 -67.0557953
Localização não encontrada BR 364 KM 90 RAMAL GRANADA KM 16 ACRELANDIA
Localização não encontrada AC 475 VILA REDENCAOI ACRELANDIA


In [ ]:
# consulta de CEP usando webservice
endereco = pycep_correios.get_address_from_cep(
    '03987-110',
    webservice=WebService.APICEP
)
print(endereco)

{'bairro': 'Vila Portuguesa', 'cep': '03987-110', 'cidade': 'São Paulo', 'logradouro': 'Rua dos Espigueiros', 'uf': 'SP', 'complemento': ''}


In [ ]:
geolocator = Nominatim(user_agent="test_app")
location = geolocator.geocode(endereco['logradouro'] + ", " + endereco['cidade'])
print(location.latitude, location.longitude)

-23.6051282 -46.5122764
